In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense, GRU, Embedding #, CuDNNGRU
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
mark_start = "xxxx "
mark_end = " yyyy"

# mark_start : Model çeviri yapmak için Decoder ile çeviri yapması gerekiyor.
                # Bunu yapabilmesi için başlangıç olarak bir cümleye ihtiyacı var. mark_start bu başlangıcı sağlar.
                # Kelimenin sonundaki boşluk çok önemli. Eğer boşluk olmazsa ürettiği kelimelerin başına xxxx koyarak anlamsız cümleler üretir.
# mark_end   : mark_start'ın cümle sonuna gelen versiyonu ve aynı kurallar geçerlidir.

In [ ]:
data_src = []   # english data
data_dest = []  # turkish data

In [ ]:
for line in open("tur.txt", encoding="UTF-8"):
  en_text, tr_text = line.rstrip().split("\t")      # rstrip : Satırdaki gereksiz boşluk karakterlerini siler.
  tr_text = mark_start + tr_text + mark_end
  data_src.append(en_text)
  data_dest.append(tr_text)

In [ ]:
data_src[100]

'I drove.'

In [ ]:
data_dest[100]

'xxxx Araba sürdüm. yyyy'

In [ ]:
data_dest[200000]


'xxxx Eksik bir şey görebiliyor musun? yyyy'

In [ ]:
len(data_src)

473035

In [ ]:
class TokenizerWrap(Tokenizer):
  def __init__(self, texts, padding, reverse=False, num_words=None):
    # texts   : tokenize yapılacak text
    # padding : Token'ların başına mı sonuna mı padding uygulanacağını belirtir.
    # reverse : truncating pre mi olacak yoksa post mu?
      # Truncating: Gelen ınput'daki fazlalık olan(input limiti haricindeki) kısmın kesilme işlemine denir.
        # pre : Token'lerin başından kesme işlemi yapılır.
        # post : Token'lerin sonunudan kesme işlemi yapılır.
      # Encoder için pre, Decoder için post kullanacağız.
    Tokenizer.__init__(self, num_words=num_words)

    self.fit_on_texts(texts)
    self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
    self.tokens = self.texts_to_sequences(texts)

    if reverse:
      self.tokens = [list(reversed(x)) for x in self.tokens]
      truncating = "pre"
    else:
      truncating = "post"


    self.num_tokens = [len(x) for x in self.tokens]
    self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
    self.max_tokens = int(self.max_tokens)

    self.tokens_padded = pad_sequences(self.tokens,
                                       maxlen = self.max_tokens,
                                       padding=padding,
                                       truncating=truncating)

  def token_to_word(self, token):
    word = " " if token == 0 else self.index_to_word[token]
    return word

  def tokens_to_string(self, tokens):
    words = [self.index_to_word[token] for token in tokens if token != 0]
    text = " ".join(words)
    return text

  def text_to_tokens(self, text, padding, reverse=False):
    tokens = self.texts_to_sequences([text])
    tokens = np.array(tokens)

    if reverse:
      tokens = np.flip(tokens, axis=1)
      truncating = "pre"
    else:
      truncating = "post"

    tokens = pad_sequences(tokens,
                           maxlen=self.max_tokens,
                           padding=padding,
                           truncating=truncating)
    return tokens


In [ ]:
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding="pre",
                              reverse=True,
                              num_words=None)

In [ ]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding="post",
                              reverse=False,
                              num_words=None)

In [ ]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded

print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


In [ ]:
tokens_dest[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0,    0],
      dtype=int32)

In [ ]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'xxxx eksik bir şey görebiliyor musun yyyy'

In [ ]:
tokens_src[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [ ]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

In [ ]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

In [ ]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [ ]:
encoder_input_data = tokens_src

In [ ]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]

In [ ]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [ ]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0], dtype=int32)

In [ ]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'xxxx eksik bir şey görebiliyor musun yyyy'

In [ ]:
tokenizer_dest.tokens_to_string(decoder_output_data[200000])

'eksik bir şey görebiliyor musun yyyy'

In [ ]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

In [ ]:
num_encoder_words

21315

In [ ]:
num_decoder_words

94058

In [ ]:
embedding_size = 100

In [ ]:
word2vec = {}
with open("glove.6B.100d.txt", encoding="UTF-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype="float32")
    word2vec[word] = vec

In [ ]:
embedding_matrix = np.random.uniform(-1,1,(num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items():
  if i < num_encoder_words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(21315, 100)

In [ ]:
encoder_input = Input(shape=(None,), name="encoder_input")

In [ ]:
encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name="encoder_embedding")

In [ ]:
state_size = 256

In [ ]:
encoder_gru1 = GRU(state_size, name="encoder_gru1", return_sequences=True)
encoder_gru2 = GRU(state_size, name="encoder_gru2", return_sequences=True)
encoder_gru3 = GRU(state_size, name="encoder_gru3", return_sequences=False)

In [ ]:
def connect_encoder():
  net = encoder_input
  net = encoder_embedding(net)
  net = encoder_gru1(net)
  net = encoder_gru2(net)
  net = encoder_gru3(net)
  encoder_output = net
  return encoder_output

In [ ]:
encoder_output = connect_encoder()

In [ ]:
decoder_initial_state = Input(shape=(state_size), name="decoder_initial_state")

In [ ]:
decoder_input = Input(shape=(None,), name="decoder_input")

In [ ]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name="decoder_embedding")

In [ ]:
decoder_gru1 = GRU(state_size, name="decoder_gru1", return_sequences=True)
decoder_gru2 = GRU(state_size, name="decoder_gru2", return_sequences=True)
decoder_gru3 = GRU(state_size, name="decoder_gru3", return_sequences=True)

In [ ]:
decoder_dense = Dense(num_decoder_words,
                      activation="linear",
                      name="decoder_output")

In [ ]:
def connect_decoder(initial_state):
  net = decoder_input
  net = decoder_embedding(net)
  net = decoder_gru1(net, initial_state=initial_state)
  net = decoder_gru2(net, initial_state=initial_state)
  net = decoder_gru3(net, initial_state=initial_state)
  decoder_output = decoder_dense(net)
  return decoder_output

In [ ]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

In [ ]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

In [ ]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [ ]:
def sparse_cross_entropy(y_true, y_pred):
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
  loss_mean = tf.reduce_mean(loss)
  return loss_mean

In [ ]:
optimizer = RMSprop(learning_rate=1e-3)

In [ ]:
#decoder_target = tf.compat.v1.placeholder(dtype="int32", shape=(None,None))
decoder_target = tf.keras.Input(shape=(None, None), dtype=tf.int32)

In [ ]:
# model_train.compile(
#     optimizer=optimizer,
#     loss=sparse_cross_entropy,
#     target_tensors=[decoder_target]
# )

model_train.compile(
    optimizer=optimizer,
    loss=sparse_cross_entropy
)

In [ ]:
path_checkpoint = "checkpoint.keras"
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [ ]:
open(path_checkpoint, 'a').close()

In [ ]:
try:
  model_train.load_weights(path_checkpoint)
except Exception as error:
  print("Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.")
  print(error)

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
Unable to open file (file signature not found)


In [ ]:
x_data = {"encoder_input" : encoder_input_data, "decoder_input" : decoder_input_data}

In [ ]:
y_data = {"decoder_output" : decoder_output_data}

In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=0,
                callbacks=[checkpoint])

In [ ]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding='pre')

    initial_state = model_encoder.predict(input_tokens)

    max_tokens = tokenizer_dest.max_tokens

    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)

    token_int = token_start
    output_text = ''
    count_tokens = 0

    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}

        decoder_output = model_decoder.predict(x_data)

        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)

        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1

    print('Input text:')
    print(input_text)
    print()

    print('Translated text:')
    print(output_text)
    print()

    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()

In [ ]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000])

1/1 [==============================] - 5s 5s/step


<ipython-input-85-6f79080a9e3f>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)


1/1 [==============================] - 0s 21ms/step
Input text:
You are telling it second hand, aren't you?

Translated text:
 yeniği ilgileneceksin ilgileneceksin istenen istenen istenen istenen birbirimiz sitesinde sitesinde

True output text:
xxxx Onu dolaylı olarak anlatıyorsun, değil mi? yyyy



In [ ]:
translate(input_text="Which road leads to the airport?")

1/1 [==============================] - 0s 19ms/step


<ipython-input-85-6f79080a9e3f>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)


1/1 [==============================] - 0s 20ms/step
Input text:
Which road leads to the airport?

Translated text:
 alacaklar götürülüyorum götürülüyorum dokunulmazlığım istenen istenen istenen usanıyorum yazardınız oluşturmadım

